# GLU Units

## Load packages, data

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import time
import gc
import os

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow import keras
from tensorflow.keras.layers import Embedding
from tensorflow.keras import activations
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras.layers.experimental.preprocessing import Normalization

from sklearn.metrics import mean_absolute_error
from datetime import datetime
import matplotlib.pyplot as plt
import pickle5 as pickle
import numpy as np
from google.cloud import bigquery
from google.cloud import storage
import gcsfs
from sklearn import preprocessing

from ficc.utils.auxiliary_functions import sqltodf
from ficc.utils.auxiliary_variables import PREDICTORS, NON_CAT_FEATURES, BINARY, CATEGORICAL_FEATURES, IDENTIFIERS, PURPOSE_CLASS_DICT, NUM_OF_DAYS_IN_YEAR
from ficc.utils.gcp_storage_functions import upload_data, download_data

import sys
sys.path.append('../')
from ficc_keras_utils import *
import ficc_keras_utils

pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(f'TF Version: {tf.__version__}')

2023-09-07 20:20:25.918932: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-07 20:20:25.931599: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-07 20:20:25.933255: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Initializing pandarallel with 16.0 cores
INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
TF Version: 2.7.0


In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/jupyter/ficc/isaac_creds.json"
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
pd.options.mode.chained_assignment = None

bq_client = bigquery.Client()
storage_client = storage.Client()

##COMMON VARIABLES
#DATA WINDOW
train_start = '2023-02-01'
train_end = '2023-07-01'
test_start = '2023-07-01'
test_end = '2023-08-01'
#MODEL PARAMETERS 
VALIDATION_SPLIT = 0.1 #ficc_keras_utils.VALIDATION_SPLIT
LEARNING_RATE = ficc_keras_utils.LEARNING_RATE
BATCH_SIZE = 1000 #ficc_keras_utils.BATCH_SIZE
NUM_EPOCHS = 75 #ficc_keras_utils.NUM_EPOCHS
DROPOUT = 0.1 #ficc_keras_utils.DROPOUT

##NOTEBOOK SPECIFIC VARIABLES 
TRADE_SEQUENCE_LENGTH = 5
NUM_FEATURES = 6
target_variable = 'new_ys' 
trade_history_col = 'trade_history_shortened'

In [3]:
def load_data_from_pickle(path, bucket = 'isaac_data'):
    if os.path.isfile(path):
        print('File available, loading pickle')
        with open(path, 'rb') as f:
            data = pickle.load(f)
    else:
        print(f'File not available, downloading from cloud storage and saving to {path}')
        fs = gcsfs.GCSFileSystem(project='eng-reactor-287421')
        gc_path = os.path.join(bucket, path)
        print(gc_path)
        with fs.open(gc_path) as gf:
            data = pd.read_pickle(gf)
        with open(path, 'wb') as f:
            pickle.dump(data, f)
    return data

In [ ]:
--bucket=
--file= 

In [5]:
%time processed_data = load_data_from_pickle('data_jan23_aug_23.pkl', bucket = 'custom-train-job-test')

File not available, downloading from cloud storage and saving to data_jan23_aug_23.pkl
custom-train-job-test/data_jan23_aug_23.pkl
CPU times: user 3min 8s, sys: 29.4 s, total: 3min 37s
Wall time: 5min 44s


In [39]:
processed_data.trade_date.min(), processed_data.trade_date.max()

(Timestamp('2023-01-03 00:00:00'), Timestamp('2023-08-15 00:00:00'))

In [40]:
processed_data.reset_index(drop=True, inplace=True)
processed_data.sort_values('trade_datetime', inplace=True)

In [41]:
def process_data(data): 
    # data['ted-rate'] = (data['t_rate_10'] - data['t_rate_2']) * 100
    
    data = data[(data.days_to_call == 0) | (data.days_to_call > np.log10(400))]
    data = data[(data.days_to_refund == 0) | (data.days_to_refund > np.log10(400))]
    data = data[(data.days_to_maturity == 0) | (data.days_to_maturity > np.log10(400))]
    data = data[data.days_to_maturity < np.log10(30000)]
    data['trade_history_sum'] = data.trade_history.parallel_apply(lambda x: np.sum(x))
    data.issue_amount = data.issue_amount.replace([np.inf, -np.inf], np.nan)
    data.dropna(inplace=True, subset=PREDICTORS+['trade_history_sum'])
    data.purpose_sub_class.fillna(0, inplace=True)
    
    return data

In [42]:
ttype_dict = { (0,0):'D', (0,1):'S', (1,0):'P' }

ys_variants = ["max_ys", "min_ys", "max_qty", "min_ago", "D_min_ago", "P_min_ago", "S_min_ago"]
ys_feats = ["_ys", "_ttypes", "_ago", "_qdiff"]
D_prev = dict()
P_prev = dict()
S_prev = dict()

def get_trade_history_columns():
    '''
    This function is used to create a list of columns
    '''
    YS_COLS = []
    for prefix in ys_variants:
        for suffix in ys_feats:
            YS_COLS.append(prefix + suffix)
    return YS_COLS

def extract_feature_from_trade(row, name, trade):
    yield_spread = trade[0]
    ttypes = ttype_dict[(trade[3],trade[4])] + row.trade_type
    seconds_ago = trade[5]
    diff = 10**trade[2] - 10**row.quantity
    quantity_diff = np.sign(diff) * np.log10(1 + np.abs(diff))
    return [yield_spread, ttypes,  seconds_ago, quantity_diff]

def trade_history_derived_features(row):
    trade_history = row.trade_history
    trade = trade_history[0]
    
    D_min_ago_t = D_prev.get(row.cusip,trade)
    D_min_ago = 9        

    P_min_ago_t = P_prev.get(row.cusip,trade)
    P_min_ago = 9
    
    S_min_ago_t = S_prev.get(row.cusip,trade)
    S_min_ago = 9
    
    max_ys_t = trade; max_ys = trade[0]
    min_ys_t = trade; min_ys = trade[0]
    max_qty_t = trade; max_qty = trade[2]
    min_ago_t = trade; min_ago = trade[5]
    
    for trade in trade_history[0:]:
        #Checking if the first trade in the history is from the same block
        if trade[5] == 0: 
            continue
 
        if trade[0] > max_ys: 
            max_ys_t = trade
            max_ys = trade[0]
        elif trade[0] < min_ys: 
            min_ys_t = trade; 
            min_ys = trade[0]

        if trade[2] > max_qty: 
            max_qty_t = trade 
            max_qty = trade[2]
        if trade[5] < min_ago: 
            min_ago_t = trade; 
            min_ago = trade[5]
            
        side = ttype_dict[(trade[3],trade[4])]
        if side == "D":
            if trade[5] < D_min_ago: 
                D_min_ago_t = trade; D_min_ago = trade[5]
                D_prev[row.cusip] = trade
        elif side == "P":
            if trade[5] < P_min_ago: 
                P_min_ago_t = trade; P_min_ago = trade[5]
                P_prev[row.cusip] = trade
        elif side == "S":
            if trade[5] < S_min_ago: 
                S_min_ago_t = trade; S_min_ago = trade[5]
                S_prev[row.cusip] = trade
        else: 
            print("invalid side", trade)
    
    trade_history_dict = {"max_ys":max_ys_t,
                          "min_ys":min_ys_t,
                          "max_qty":max_qty_t,
                          "min_ago":min_ago_t,
                          "D_min_ago":D_min_ago_t,
                          "P_min_ago":P_min_ago_t,
                          "S_min_ago":S_min_ago_t}

    return_list = []
    for variant in ys_variants:
        feature_list = extract_feature_from_trade(row,variant,trade_history_dict[variant])
        return_list += feature_list
    
    return return_list

In [43]:
%%time
temp = processed_data[['cusip','trade_history','quantity','trade_type']].parallel_apply(trade_history_derived_features, axis=1)
YS_COLS = get_trade_history_columns()
processed_data[YS_COLS] = pd.DataFrame(temp.tolist(), index=processed_data.index)

CPU times: user 2min 53s, sys: 23.8 s, total: 3min 17s
Wall time: 5min 55s


In [44]:
%%time
processed_data['ttypes'] = (processed_data.last_trade_type.astype(str) + processed_data.trade_type.astype(str)).astype('category')
processed_data['diff_size'] = (processed_data.par_traded.astype(float) - processed_data.last_size).astype(np.float32)
processed_data['abs_last_yield_spread'] = np.abs(processed_data['last_yield_spread'])
processed_data['abs_diff_size'] = np.abs(processed_data['diff_size'])
processed_data['days_duration'] = (processed_data.last_calc_date - processed_data.last_settlement_date).dt.days

CPU times: user 1.55 s, sys: 117 ms, total: 1.67 s
Wall time: 1.67 s


In [45]:
%%time
processed_data['trade_history_sum'] = processed_data.trade_history.parallel_apply(lambda x: np.sum(x))
processed_data = processed_data.dropna(subset=['trade_history_sum'])
processed_data.purpose_sub_class.fillna(0, inplace=True)

CPU times: user 44.8 s, sys: 15.2 s, total: 1min
Wall time: 1min 7s


In [46]:
for col in ['new_ficc_ycl', 'yield_spread']:
    if col in NON_CAT_FEATURES:
        NON_CAT_FEATURES.remove(col)
    if col in PREDICTORS:
        PREDICTORS.remove(col)

In [47]:
%%time 
processed_data = process_data(processed_data)

CPU times: user 1min 11s, sys: 19.6 s, total: 1min 30s
Wall time: 1min 38s


In [48]:
processed_data['moodys_rating'] = processed_data.moodys_long.fillna('NR')
processed_data['ratings_concat'] = processed_data['rating'] + processed_data['moodys_rating']

In [49]:
def modify_features(cols, how, where=None):
    #function to conveniently add or subtract features from the global feature lists 
    
    global CATEGORICAL_FEATURES, PREDICTORS, NON_CAT_FEATURES, BINARY
    
    if how not in ['add','remove']:
        raise ValueError(f"'how' argument must be one off the following: {['add','remove']}")
        
    if where and where not in ['categorical','binary', 'numeric']:
        raise ValueError(f"'where' argument must be one off the following: {['categorical','binary', 'numeric']}")
    
    if how == 'remove':
        for col in cols: 
            if col in CATEGORICAL_FEATURES:
                CATEGORICAL_FEATURES.remove(col)
            if col in BINARY:
                BINARY.remove(col)
            if col in NON_CAT_FEATURES:
                NON_CAT_FEATURES.remove(col)
            if col in PREDICTORS:
                PREDICTORS.remove(col)
            
    if how == 'add':
        for col in cols: 
            if col not in CATEGORICAL_FEATURES and where=='categorical':
                CATEGORICAL_FEATURES.append(col)
            if col not in BINARY and where=='binary':
                BINARY.append(col)
            if col not in NON_CAT_FEATURES and where=='numeric':
                NON_CAT_FEATURES.append(col)
            if col not in PREDICTORS:
                PREDICTORS.append(col)

In [50]:
additional_features = ['ttypes', 'diff_size', 'abs_last_yield_spread', 'abs_diff_size', 'days_duration']
YS_COLS = ['max_ys_ys', 'max_ys_ttypes', 'max_ys_ago', 'max_ys_qdiff', 'min_ys_ys', 'min_ys_ttypes', 'min_ys_ago', 'min_ys_qdiff', 'max_qty_ys', 'max_qty_ttypes', \
           'max_qty_ago', 'max_qty_qdiff', 'min_ago_ys', 'min_ago_ttypes', 'min_ago_ago', 'min_ago_qdiff', 'D_min_ago_ys', 'D_min_ago_ttypes', 'D_min_ago_ago', 'D_min_ago_qdiff',\
           'P_min_ago_ys', 'P_min_ago_ttypes', 'P_min_ago_ago', 'P_min_ago_qdiff', 'S_min_ago_ys', 'S_min_ago_ttypes', 'S_min_ago_ago', 'S_min_ago_qdiff']
    
for col in YS_COLS:
    if 'ttypes' in col and col not in PREDICTORS:
        PREDICTORS.append(col)
        CATEGORICAL_FEATURES.append(col)
    elif col not in PREDICTORS:
        NON_CAT_FEATURES.append(col)
        PREDICTORS.append(col)
        
if 'target_attention_features' not in PREDICTORS:
    PREDICTORS.append('target_attention_features')
    
if 'ficc_treasury_spread' not in PREDICTORS:
    PREDICTORS.append('ficc_treasury_spread')
    NON_CAT_FEATURES.append('ficc_treasury_spread')

#add for the encoders, then remove later if needed 
modify_features(['rating','moodys_rating', 'ratings_concat'], how = 'add', where='categorical')   
    
encoders = {}
fmax = {}
for f in CATEGORICAL_FEATURES:
    print(f)
    if f in ['rating', 'incorporated_state_code', 'trade_type', 'purpose_class']:
        fprep = preprocessing.LabelEncoder().fit(categorical_feature_values[f])
    else:
        fprep = preprocessing.LabelEncoder().fit(processed_data[f].drop_duplicates())
    fmax[f] = np.max(fprep.transform(fprep.classes_))
    encoders[f] = fprep
    
# with open('encoders.pkl','wb') as file:
#     pickle.dump(encoders,file)

rating
incorporated_state_code
trade_type
purpose_class
max_ys_ttypes
min_ys_ttypes
max_qty_ttypes
min_ago_ttypes
D_min_ago_ttypes
P_min_ago_ttypes
S_min_ago_ttypes
moodys_rating
ratings_concat


In [55]:
%%time
TRADE_SEQUENCE_LENGTH = 5 
processed_data['trade_history_shortened'] = processed_data['trade_history'].apply(lambda x: x[:TRADE_SEQUENCE_LENGTH])

CPU times: user 7.23 s, sys: 360 ms, total: 7.59 s
Wall time: 7.71 s


In [56]:
processed_data.sort_values('trade_datetime',ascending=False,inplace=True)

In [87]:
train_filter = (processed_data.trade_date < train_end) & (processed_data.trade_date >= train_start)
test_filter = (processed_data.trade_date >= test_start) & (processed_data.trade_date <test_end)
                                                            
train_dataframe = processed_data[train_filter]\
.sort_values(by='trade_date', ascending=True)\
.reset_index(drop=True)

test_dataframe = processed_data[test_filter]\
.sort_values(by='trade_date', ascending=True)\
.reset_index(drop=True)

print('Training data start: {}, end: {}'.format(train_dataframe.trade_date.min(),train_dataframe.trade_date.max()))
print('Test data start: {}, end: {}'.format(test_dataframe.trade_date.min(),test_dataframe.trade_date.max()))

Training data start: 2023-02-01 00:00:00, end: 2023-06-30 00:00:00
Test data start: 2023-07-03 00:00:00, end: 2023-07-31 00:00:00


In [88]:
len(train_dataframe), len(test_dataframe)

(3387004, 643018)

In [89]:
def create_input_new(df, trade_history_col, yield_history_cols):
    global encoders
    datalist = []
    
    datalist.append(np.stack(df[trade_history_col].to_numpy()))
    datalist.append(np.stack(df['target_attention_features'].to_numpy()))

    noncat_and_binary = []
    for f in NON_CAT_FEATURES + BINARY:
        noncat_and_binary.append(np.expand_dims(df[f].to_numpy().astype('float32'), axis=1))
    datalist.append(np.concatenate(noncat_and_binary, axis=-1))
    
    for f in CATEGORICAL_FEATURES:
        encoded = encoders[f].transform(df[f])
        datalist.append(encoded.astype('float32'))
    
    return datalist

def create_data_set_and_model(train_dataframe, test_dataframe, trade_history_col, yield_history_cols):
    
    if not isinstance(trade_history_col, str):
        raise ValueError('trade_history_col must be a string')
    
    if isinstance(yield_history_cols, str):
        num_yield_history = 1
        yield_history_cols = [yield_history_cols]
    else:
        num_yield_history = len(yield_history_cols)
    
    yield_history_lengths = [train_dataframe[x][0].shape[0] for x in yield_history_cols]
    
    TRADE_SEQUENCE_LENGTH = train_dataframe[trade_history_col][0].shape[0] 
    
    params = {'TRADE_SEQUENCE_LENGTH':TRADE_SEQUENCE_LENGTH, 
           'yield_history_cols':yield_history_cols, 
           'yield_history_lengths':yield_history_lengths, 
           'num_yield_history':num_yield_history }
    
    np.random.seed(1)
    val_idx = np.random.choice(range(len(train_dataframe)), 
                     size = int(VALIDATION_SPLIT*len(train_dataframe)),
                     replace=False)

    print(f'TRAINING DATA: N = {len(train_dataframe)-len(val_idx)}, MIN DATE = {train_dataframe.drop(val_idx, axis=0).trade_date.min()}, MAX DATE = {train_dataframe.drop(val_idx, axis=0).trade_date.max()}')
    print(f'VALIDATION DATA: N = {len(val_idx)}, MIN DATE = {train_dataframe.iloc[val_idx].trade_date.min()}, MAX DATE = {train_dataframe.iloc[val_idx].trade_date.max()}')
    print(f'TEST DATA: N = {len(test_dataframe)}, MIN DATE = {test_dataframe.trade_date.min()}, MAX DATE = {test_dataframe.trade_date.max()}')

    x_train = create_input_new(train_dataframe.drop(val_idx, axis=0), trade_history_col, yield_history_cols)
    y_train = train_dataframe.drop(val_idx, axis=0)[target_variable]

    x_val = create_input_new(train_dataframe.iloc[val_idx], trade_history_col, yield_history_cols)
    y_val = train_dataframe.iloc[val_idx][target_variable]

    x_test = create_input_new(test_dataframe, trade_history_col, yield_history_cols)
    y_test = test_dataframe[target_variable]    
    
    # Normalization layer for the yield history
    yield_history_normalizers = []
    for i in range(num_yield_history):
        yield_history_normalizers.append(Normalization(name=f'Yield_history_normalizer_{yield_history_cols[i]}'))
        yield_history_normalizers[i].adapt(x_train[i],batch_size=BATCH_SIZE)
    
    with tf.device('/cpu:0'):
        # Normalization layer for the trade history
        trade_history_normalizer = Normalization(name='Trade_history_normalizer')
        trade_history_normalizer.adapt(x_train[0],batch_size=BATCH_SIZE)

        # Normalization layer for the non-categorical and binary features
        noncat_binary_normalizer = Normalization(name='Numerical_binary_normalizer')
        noncat_binary_normalizer.adapt(x_train[2], batch_size = BATCH_SIZE)

    normalizers = {'yield_history_normalizers': yield_history_normalizers,
                  'trade_history_normalizer': trade_history_normalizer,
                  'noncat_binary_normalizer': noncat_binary_normalizer}

    return  params, normalizers, x_train, y_train, x_val, y_val, x_test, y_test, val_idx

def create_tf_data(x_train, y_train, shuffle=False, shuffle_buffer=1):
                     
    X=()
    for x in x_train:
        X += (tf.data.Dataset.from_tensor_slices(x),)
        

    temp = tf.data.Dataset.zip((X))
    del X
    dataset = tf.data.Dataset.zip((temp,
                        tf.data.Dataset.from_tensor_slices(y_train)))
    del temp
    if shuffle:
        shuffle_buffer = int(len(x_train[0])*shuffle_buffer)
        dataset = dataset.shuffle(shuffle_buffer)
        
    return dataset


#!pip install gpustat
from tensorflow.keras.callbacks import Callback
import regex as r
import gpustat

class LossCallback(Callback):
    def on_epoch_begin(self, batch, logs={}):
        super().on_epoch_begin(batch)
        self.gpu_usage = []
        
    def on_train_batch_end(self, batch, logs={}):  
        if batch % 25 == 0:
            s = str(gpustat.new_query()[0])
            gpu_usage = int(r.findall('[0-9]+ %', s)[0].split('%')[0])
            self.gpu_usage.append(gpu_usage)
        
    def on_epoch_end(self, epoch, logs=None):
        s = str(gpustat.new_query()[0])
        gpu_mem = r.findall('[0-9]+ / [0-9]+', s)[0].replace(' ', '')
        
        if logs is None:
            logs = {}
            
        train_loss = []
        val_loss = []
        
        
        train_loss_cols = [x for x in logs.keys() if 'val' not in x]
        val_loss_cols = [x for x in logs.keys() if 'val' in x]
        
        for loss in train_loss_cols:
            
                train_loss.append(logs.get(loss, None))
                
        for loss in val_loss_cols:
            
                val_loss.append(logs.get(loss, None))
        
        if train_loss and val_loss:
            # print(f'Epoch {epoch + 1}')
            df = pd.DataFrame([train_loss, val_loss], 
                              index = ['Train loss', 'Val loss'],
                              # columns = [f'Ensemble_model_{i+1}' for i in range(ENSEMBLE_SIZE)],
                             columns = train_loss_cols)
            
            df['GPU MEMORY'] = gpu_mem
            df['MAX GPU USAGE'] = str(max(self.gpu_usage))+'%' if self.gpu_usage else None
            display(df)
            print()
            
loss_callback = LossCallback()

fit_callbacks = [
keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    verbose=0,
    mode="auto",
    restore_best_weights=True),
    # CSVLoggerTimeHistory(timestamp+'_training_logs_yield_history.csv', separator=",", append=False),
    loss_callback
]

In [90]:
trade_history_col = 'trade_history_shortened'
yield_history_cols = []
modify_features(['moodys_rating', 'ratings_concat'], how = 'remove')   

params, normalizers, x_train, y_train, x_val, y_val, x_test, y_test, val_idx = create_data_set_and_model(train_dataframe,                                                                                                         
                                                                                                         test_dataframe, 
                                                                                                         trade_history_col, 
                                                                                                         yield_history_cols)

TRAINING DATA: N = 3048304, MIN DATE = 2023-02-01 00:00:00, MAX DATE = 2023-06-30 00:00:00
VALIDATION DATA: N = 338700, MIN DATE = 2023-02-01 00:00:00, MAX DATE = 2023-06-30 00:00:00
TEST DATA: N = 643018, MIN DATE = 2023-07-03 00:00:00, MAX DATE = 2023-07-31 00:00:00


In [91]:
print(len(x_train))
for i in range(len(x_train)):
    print(x_train[i].shape)

14
(3048304, 5, 6)
(3048304, 1, 3)
(3048304, 48)
(3048304,)
(3048304,)
(3048304,)
(3048304,)
(3048304,)
(3048304,)
(3048304,)
(3048304,)
(3048304,)
(3048304,)
(3048304,)


In [92]:
shuffle = True
shuffle_buffer = 0.75

TRADE_SEQUENCE_LENGTH = params.get('TRADE_SEQUENCE_LENGTH')
yield_history_cols = params.get('yield_history_cols')
yield_history_lengths = params.get('yield_history_lengths')
num_yield_history = params.get('num_yield_history')

yield_history_normalizers = normalizers.get('yield_history_normalizers')
trade_history_normalizer = normalizers.get('trade_history_normalizer')
noncat_binary_normalizer = normalizers.get('noncat_binary_normalizer')

tf.keras.utils.set_random_seed(10)

timestamp = datetime.now().strftime('%Y-%m-%d %H-%M')

with tf.device('/cpu:0'):
    train_ds = create_tf_data(x_train, y_train, shuffle, shuffle_buffer)
    train_ds = train_ds.batch(BATCH_SIZE).prefetch(2).cache()
    val_ds = create_tf_data(x_val, y_val, shuffle = False)
    val_ds = val_ds.batch(5000).prefetch(2).cache()

In [93]:
class GatedLinearUnit(keras.layers.Layer):
    def __init__(self):
        super().__init__()
    
    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1]),
                                 initializer='glorot_uniform',
                                 trainable=True,
                                name = 'W')
        
        self.bias1 = self.add_weight(
            shape=(input_shape[-1],),
            initializer="zeros",
            trainable=True,
                                name = 'b1')
        
        self.V = self.add_weight(shape=(input_shape[-1], input_shape[-1]),
                                 initializer='glorot_uniform',
                                 trainable=True,
                                 name = 'V')
        self.bias2 = self.add_weight(
            shape=(input_shape[-1],),
            initializer="zeros",
            trainable=True,
        name='b2')

    def call(self, inputs):
        sigmoid = tf.sigmoid(inputs @ self.W + self.bias1)
        output = tf.multiply(inputs @ self.V + self.bias2, sigmoid)
        return output

In [94]:
def generate_model(TRADE_SEQUENCE_LENGTH, yield_history_cols, yield_history_lengths, num_yield_history,
                      yield_history_normalizers, trade_history_normalizer, noncat_binary_normalizer):
    inputs = []
    layer = []
    
    trade_history_input = layers.Input(name="trade_history_input", 
                                       shape=(TRADE_SEQUENCE_LENGTH, NUM_FEATURES), 
                                       dtype = tf.float32) 

    target_attention_input = layers.Input(name="target_attention_input", 
                                       shape=(1, 3), 
                                       dtype = tf.float32) 
    inputs.append(trade_history_input)
    inputs.append(target_attention_input)

    inputs.append(layers.Input(
        name="NON_CAT_AND_BINARY_FEATURES",
        shape=(len(NON_CAT_FEATURES + BINARY),)
    ))


    layer.append(noncat_binary_normalizer(inputs[num_yield_history+2]))
    ####################################################


    ############## TRADE HISTORY MODEL #################

    lstm_layer = layers.Bidirectional(layers.LSTM(50, 
                             activation='tanh',
                             input_shape=(TRADE_SEQUENCE_LENGTH,NUM_FEATURES),
                             return_sequences = True,
                             name='LSTM'))

    lstm_layer_2 = layers.Bidirectional(layers.LSTM(100, 
                                                    activation='tanh',
                                                    input_shape=(TRADE_SEQUENCE_LENGTH, 50),
                                                    return_sequences = True,
                                                    name='LSTM_2'))



    features = lstm_layer(trade_history_normalizer(inputs[num_yield_history]))
    features = lstm_layer_2(features)  
    
    
    attention_sequence = layers.Dense(200, activation='relu', name='attention_dense')(target_attention_input)
    attention = layers.Dot(axes=[2, 2])([features, attention_sequence])
    attention = layers.Activation('softmax')(attention)

    context_vector = layers.Dot(axes=[1, 1])([features, attention])
    context_vector = layers.Flatten(name='context_vector_flatten')(context_vector)


    trade_history_output = layers.Dense(100, 
                                        activation='relu')(context_vector)
 
    ####################################################

    ############## REFERENCE DATA MODEL ################
    global encoders
    for f in CATEGORICAL_FEATURES:
        fin = layers.Input(shape=(1,), name = f)
        inputs.append(fin)
        embedded = layers.Flatten(name = f + "_flat")( layers.Embedding(input_dim = fmax[f]+1,
                                                                        output_dim = max(30,int(np.sqrt(fmax[f]))),
                                                                        input_length= 1,
                                                                        name = f + "_embed")(fin))
        layer.append(embedded)
    
    
    ### CONCATENATE EMBEDDINGS
    reference_hidden = layers.Dropout(DROPOUT)(layers.concatenate(layer, axis=-1))
    reference_hidden = layers.Dense(400,
                                    activation='relu',
                                    name='reference_hidden_1')(reference_hidden)
    reference_hidden = layers.BatchNormalization()(reference_hidden)
    reference_GLU = GatedLinearUnit()(reference_hidden)
    

    reference_hidden2 = layers.Dropout(DROPOUT)(reference_GLU)
    reference_hidden2 = layers.Dense(200,activation='relu',name='reference_hidden_2')(reference_hidden2)
    reference_hidden2 = layers.BatchNormalization()(reference_hidden2)
    reference_GLU2 = GatedLinearUnit()(reference_hidden2)
    
    
    reference_hidden3 = layers.Dropout(DROPOUT)(reference_GLU2)
    reference_hidden3 = layers.Dense(100,activation='relu',name='reference_hidden_3')(reference_hidden3)
    reference_hidden3 = layers.BatchNormalization()(reference_hidden3)
    reference_output = GatedLinearUnit()(reference_hidden3)
   

    ####################################################

    feed_forward_input = layers.concatenate([reference_output, trade_history_output])
    
    hidden = layers.Dropout(DROPOUT)(feed_forward_input)
    hidden = layers.Dense(300,activation='relu')(feed_forward_input)
    hidden = layers.BatchNormalization()(hidden)
    hidden = GatedLinearUnit()(hidden)

    hidden2 = layers.Dropout(DROPOUT)(hidden)
    hidden2 = layers.Dense(100,activation='relu')(hidden)
    hidden2 = layers.BatchNormalization()(hidden2)
    hidden2 = GatedLinearUnit()(hidden2)

    final = layers.Dense(1)(hidden2)

    model = keras.Model(inputs=inputs, outputs=final)
    
    return model

def create_data_set_and_model(train_dataframe, test_dataframe, trade_history_col, yield_history_cols):
    
    if not isinstance(trade_history_col, str):
        raise ValueError('trade_history_col must be a string')
    
    if isinstance(yield_history_cols, str):
        num_yield_history = 1
        yield_history_cols = [yield_history_cols]
    else:
        num_yield_history = len(yield_history_cols)
    
    yield_history_lengths = [train_dataframe[x][0].shape[0] for x in yield_history_cols]
    
    TRADE_SEQUENCE_LENGTH = train_dataframe[trade_history_col][0].shape[0] 
    
    params = {'TRADE_SEQUENCE_LENGTH':TRADE_SEQUENCE_LENGTH, 
           'yield_history_cols':yield_history_cols, 
           'yield_history_lengths':yield_history_lengths, 
           'num_yield_history':num_yield_history }
    
    np.random.seed(1)
    val_idx = np.random.choice(range(len(train_dataframe)), 
                     size = int(VALIDATION_SPLIT*len(train_dataframe)),
                     replace=False)

    print(f'TRAINING DATA: N = {len(train_dataframe)-len(val_idx)}, MIN DATE = {train_dataframe.drop(val_idx, axis=0).trade_date.min()}, MAX DATE = {train_dataframe.drop(val_idx, axis=0).trade_date.max()}')
    print(f'VALIDATION DATA: N = {len(val_idx)}, MIN DATE = {train_dataframe.iloc[val_idx].trade_date.min()}, MAX DATE = {train_dataframe.iloc[val_idx].trade_date.max()}')
    print(f'TEST DATA: N = {len(test_dataframe)}, MIN DATE = {test_dataframe.trade_date.min()}, MAX DATE = {test_dataframe.trade_date.max()}')

    x_train = create_input_new(train_dataframe.drop(val_idx, axis=0), trade_history_col, yield_history_cols)
    y_train = train_dataframe.drop(val_idx, axis=0)[target_variable]

    x_val = create_input_new(train_dataframe.iloc[val_idx], trade_history_col, yield_history_cols)
    y_val = train_dataframe.iloc[val_idx][target_variable]

    x_test = create_input_new(test_dataframe, trade_history_col, yield_history_cols)
    y_test = test_dataframe[target_variable]    
    
    # Normalization layer for the yield history
    yield_history_normalizers = []
    for i in range(num_yield_history):
        yield_history_normalizers.append(Normalization(name=f'Yield_history_normalizer_{yield_history_cols[i]}'))
        yield_history_normalizers[i].adapt(x_train[i],batch_size=BATCH_SIZE)
    
    with tf.device('/cpu:0'):
        # Normalization layer for the trade history
        trade_history_normalizer = Normalization(name='Trade_history_normalizer')
        trade_history_normalizer.adapt(x_train[0],batch_size=BATCH_SIZE)

        # Normalization layer for the non-categorical and binary features
        noncat_binary_normalizer = Normalization(name='Numerical_binary_normalizer')
        noncat_binary_normalizer.adapt(x_train[2], batch_size = BATCH_SIZE)

    normalizers = {'yield_history_normalizers': yield_history_normalizers,
                  'trade_history_normalizer': trade_history_normalizer,
                  'noncat_binary_normalizer': noncat_binary_normalizer}

    return  params, normalizers, x_train, y_train, x_val, y_val, x_test, y_test, val_idx

In [95]:
def train_model_new(params, normalizers, x_train, y_train, x_val, y_val, shuffle, shuffle_buffer=1):
    tf.keras.backend.clear_session()
    gc.collect()

    TRADE_SEQUENCE_LENGTH = params.get('TRADE_SEQUENCE_LENGTH')
    yield_history_cols = params.get('yield_history_cols')
    yield_history_lengths = params.get('yield_history_lengths')
    num_yield_history = params.get('num_yield_history')
      
    yield_history_normalizers = normalizers.get('yield_history_normalizers')
    trade_history_normalizer = normalizers.get('trade_history_normalizer')
    noncat_binary_normalizer = normalizers.get('noncat_binary_normalizer')
       
    tf.keras.utils.set_random_seed(10)
    model = generate_model(TRADE_SEQUENCE_LENGTH, yield_history_cols, yield_history_lengths, num_yield_history, 
                               yield_history_normalizers, trade_history_normalizer, noncat_binary_normalizer)
    
    timestamp = datetime.now().strftime('%Y-%m-%d %H-%M')
    
    with tf.device('/cpu:0'):
        train_ds = create_tf_data(x_train, y_train, shuffle, shuffle_buffer)
        train_ds = train_ds.batch(BATCH_SIZE).prefetch(2).cache()
        val_ds = create_tf_data(x_val, y_val, shuffle = False)
        val_ds = val_ds.batch(BATCH_SIZE).prefetch(2).cache()
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
          loss=keras.losses.MeanAbsoluteError())

    history= model.fit(train_ds,
                                      validation_data=val_ds,
                                        epochs=NUM_EPOCHS,     
                                        verbose=1, 
                                        callbacks=fit_callbacks,
                                        use_multiprocessing=True,
                                        workers=8)
    
    return history, model

In [96]:
tf.keras.backend.clear_session()

In [ ]:
# results = []
# predictions = []
NUM_EPOCHS = 125

for i in range(4):
    tf.keras.utils.set_random_seed(10)
    start = time.time()
    history, model = train_model_new(params, normalizers, x_train, y_train, x_val, y_val, True, shuffle_buffer = .75)

    pred = model.predict(x_test)
    predictions.append(pred)
    print('='*25+f' TRIAL {i}, MAE: {mean_absolute_error(pred,y_test)} '+'='*25)
    results.append([history, model])

Epoch 1/125


2023-09-05 22:46:57.268644: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla T4" frequency: 1590 num_cores: 40 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 14254866432 bandwidth: 320064000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2023-09-05 22:47:09.029135: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 346170 of 2286228
2023-09-05 22:47:19.029150: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 648405 of 2286228
2023-09-05 22

3049/3049 [==============================] - ETA: 0s - loss: 14.3787

2023-09-05 22:50:09.820693: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla T4" frequency: 1590 num_cores: 40 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 14254866432 bandwidth: 320064000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


,loss,GPU MEMORY,MAX GPU USAGE
Train loss,14.379,7070/15360,92%
Val loss,10.244,7070/15360,92%


...


3049/3049 [==============================] - 113s 37ms/step - loss: 9.4353 - val_loss: 9.3210
Epoch 9/125
3047/3049 [============================>.] - ETA: 0s - loss: 9.3692

,loss,GPU MEMORY,MAX GPU USAGE
Train loss,9.369,7070/15360,94%
Val loss,9.166,7070/15360,94%



3049/3049 [==============================] - 113s 37ms/step - loss: 9.3692 - val_loss: 9.1663
Epoch 10/125
3048/3049 [============================>.] - ETA: 0s - loss: 9.3064

,loss,GPU MEMORY,MAX GPU USAGE
Train loss,9.306,7070/15360,94%
Val loss,9.064,7070/15360,94%



3049/3049 [==============================] - 114s 37ms/step - loss: 9.3064 - val_loss: 9.0645
Epoch 11/125
1422/3049 [============>.................] - ETA: 47s - loss: 9.3202

In [294]:
pred_filter = (-5000 < test_dataframe['prediction']) & (test_dataframe['prediction'] < 5000)

for i, pred in enumerate(predictions): 
    print(f'Trial {i+1}: {mean_absolute_error(pred[pred_filter], y_test[pred_filter]):.2f} ({len(results[i][0].history["val_loss"])} epochs)')

Trial 1: 7.89 (76 epochs)
Trial 2: 8.02 (125 epochs)
Trial 3: 8.04 (125 epochs)
Trial 4: 7.94 (92 epochs)
Trial 5: 8.07 (125 epochs)


In [ ]:
pred = model.predict(x_test, batch_size=5000)

In [ ]:
test_dataframe['prediction'] = pred
test_dataframe['error'] = test_dataframe.new_ys - test_dataframe['prediction']

In [136]:
mean_absolute_error(pred[pred_filter], y_test[pred_filter])

7.886971382603239

In [ ]:
for col in NON_CAT_FEATURES:
    print(f'{col}: {test_dataframe[test_dataframe.rtrs_control_number == 2023071804421200][col].values}')

quantity: [4.30103]
days_to_maturity: [3.96553094]
days_to_call: [3.14050804]
coupon: [3.875]
issue_amount: [7.9439397]
last_seconds_ago: [19942088.]
last_yield_spread: [55.16586272]
days_to_settle: [2]
days_to_par: [3.14050804]
maturity_amount: [6.59328618]
issue_price: [100.]
orig_principal_amount: [6.59328618]
max_amount_outstanding: [6.59328618]
accrued_days: [1891]
days_in_interest_payment: [180.]
A/E: [0.44444444]
max_ys_ys: [78.37144734]
max_ys_ago: [7.3017921]
max_ys_qdiff: [0.]
min_ys_ys: [40.86586272]
min_ys_ago: [7.29977065]
min_ys_qdiff: [0.]
max_qty_ys: [55.16586272]
max_qty_ago: [7.29977065]
max_qty_qdiff: [0.]
min_ago_ys: [55.16586272]
min_ago_ago: [7.29977065]
min_ago_qdiff: [0.]
D_min_ago_ys: [55.16586272]
D_min_ago_ago: [7.29977065]
D_min_ago_qdiff: [0.]
P_min_ago_ys: [78.37144734]
P_min_ago_ago: [7.3017921]
P_min_ago_qdiff: [0.]
S_min_ago_ys: [40.86586272]
S_min_ago_ago: [7.29977065]
S_min_ago_qdiff: [0.]
ficc_treasury_spread: [12.91293769]


In [ ]:
test_dataframe[test_dataframe.cusip == '64987DJC9'][['rtrs_control_number', 'trade_datetime', 'last_trade_datetime', 'cusip','new_ys', 'last_yield', 'new_real_time_ficc_ycl', 'prediction', 'error', trade_history_col]]

,rtrs_control_number,trade_datetime,last_trade_datetime,cusip,new_ys,last_yield,new_real_time_ficc_ycl,prediction,error,trade_history_shortened
348245,2023071804421200,2023-07-18 12:13:07,2022-11-29 16:44:59,64987DJC9,61.706,480.500,411.293,-10102020.000,10102081.706,"[[55.16586271742739, 99.5, 4.301030158996582, ..."
348247,2023071804420700,2023-07-18 12:13:07,2022-11-29 16:44:59,64987DJC9,66.606,480.500,411.293,-10102021.000,10102087.606,"[[55.16586271742739, 99.5, 4.301030158996582, ..."
348615,2023071804568200,2023-07-18 12:16:36,2023-07-18 12:13:07,64987DJC9,40.687,468.400,408.313,-10102006.000,10102046.687,"[[64.4870623097533, 77.4, 4.301030158996582, 0..."
348622,2023071804532000,2023-07-18 12:16:30,2023-07-18 12:13:07,64987DJC9,40.087,468.400,408.313,-10102010.000,10102050.087,"[[64.4870623097533, 77.4, 4.301030158996582, 0..."
379199,2023071900295300,2023-07-19 08:37:40,2023-07-18 12:16:36,64987DJC9,34.489,444.600,405.571,-10102019.000,10102053.489,"[[40.68706230975323, 53.6, 4.301030158996582, ..."
379418,2023071900319500,2023-07-19 08:40:02,2023-07-19 08:37:40,64987DJC9,36.202,435.000,405.558,-10102022.000,10102058.202,"[[34.50169568948314, 51.0, 4.301030158996582, ..."
399923,2023072011145200,2023-07-20 16:05:01,2023-07-19 08:40:02,64987DJC9,17.738,436.700,404.827,-10102017.000,10102034.738,"[[36.20169568948319, 52.7, 4.301030158996582, ..."


In [ ]:
test_dataframe[['cusip','new_ys', 'last_yield', 'prediction', 'error', 'coupon', 'days_to_maturity', 'default_indicator']].sort_values(by='error', key=abs).tail(25)

,cusip,new_ys,last_yield,prediction,error,coupon,days_to_maturity,default_indicator
545293,236887AW5,852.580,421.700,267.265,585.315,4.000,3.184,False
545291,236887AW5,852.380,421.700,262.808,589.572,4.000,3.184,False
375668,132047BX8,1502.105,1440.200,886.736,615.369,4.500,3.476,False
375651,132047BX8,1512.405,1440.200,886.736,625.669,4.500,3.476,False
375653,132047BX8,1512.405,1440.200,886.736,625.669,4.500,3.476,False
375650,132047BX8,1516.505,1440.200,877.017,639.487,4.500,3.476,False
497919,132047BX8,1604.270,1806.000,888.239,716.031,4.500,3.476,False
579588,227235AT5,1003.795,400.000,198.692,805.103,3.600,3.137,False
579589,227235AT5,999.395,400.000,181.047,818.347,3.600,3.137,False
121726,261333ET1,432.622,702.400,1350.019,-917.397,5.000,3.036,False


In [ ]:
problem_cusips = ['64987DHP2', '64987DHZ0', '64987DJC9']
problem_cusips_df = test_dataframe[test_dataframe.cusip.isin(problem_cusips)]
same_series = test_dataframe[~test_dataframe.cusip.isin(problem_cusips)][test_dataframe.cusip.str[:6] == '64987D']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
non_cat_comp = pd.concat([same_series[NON_CAT_FEATURES].describe().loc[['mean']],
          problem_cusips_df[NON_CAT_FEATURES].describe().loc[['mean']]]
         )

In [200]:
display(non_cat_comp.iloc[:, :20])

display(non_cat_comp.iloc[:, 20:])

,quantity,days_to_maturity,days_to_call,coupon,issue_amount,last_seconds_ago,last_yield_spread,days_to_settle,days_to_par,maturity_amount,issue_price,orig_principal_amount,max_amount_outstanding,accrued_days,days_in_interest_payment,A/E,max_ys_ys,max_ys_ago,max_ys_qdiff,min_ys_ys
mean,4.598,3.443,1.688,2.525,8.202,2188757.143,84.966,2.950,1.688,6.744,100.000,6.744,6.744,1430.283,180.000,0.442,99.190,5.111,0.671,69.592
mean,4.766,4.302,3.444,4.215,7.458,12060410.105,46.040,2.903,3.444,6.178,76.500,6.178,6.178,1420.497,136.500,1.837,57.856,6.131,2.852,38.996


,min_ys_ago,min_ys_qdiff,max_qty_ys,max_qty_ago,max_qty_qdiff,min_ago_ys,min_ago_ago,min_ago_qdiff,D_min_ago_ys,D_min_ago_ago,D_min_ago_qdiff,P_min_ago_ys,P_min_ago_ago,P_min_ago_qdiff,S_min_ago_ys,S_min_ago_ago,S_min_ago_qdiff,ficc_treasury_spread
mean,5.393,0.372,82.086,5.080,2.254,84.966,4.402,0.649,85.703,4.616,0.530,99.213,5.462,1.965,74.073,5.391,0.173,-90.305
mean,6.156,2.449,44.177,6.034,2.852,46.040,5.723,2.268,46.040,5.723,2.268,55.332,5.878,2.268,39.657,6.825,2.852,-19.571


In [218]:
pd.concat([same_series[BINARY].describe().loc[['top']],
          problem_cusips_df[BINARY].describe().loc[['top']]]
         )

,callable,sinking,zerocoupon,is_non_transaction_based_compensation,is_general_obligation,callable_at_cav,extraordinary_make_whole_call,make_whole_call,has_unexpired_lines_of_credit,escrow_exists
top,True,False,False,False,False,False,False,False,False,False
top,True,True,False,False,False,False,False,False,True,False


In [217]:
pd.concat([same_series[CATEGORICAL_FEATURES].describe().loc[['top']],
          problem_cusips_df[CATEGORICAL_FEATURES].describe().loc[['top']]]
         )

,rating,incorporated_state_code,trade_type,purpose_class,max_ys_ttypes,min_ys_ttypes,max_qty_ttypes,min_ago_ttypes,D_min_ago_ttypes,P_min_ago_ttypes,S_min_ago_ttypes
top,MR,NY,D,18.000,DD,SD,DD,DD,DD,PD,SD
top,MR,NY,D,18.000,PD,SD,DD,DD,DD,PD,SD


In [240]:
hypothetical_df = problem_cusips_df.copy()
hypothetical_df['sinking'] = False
hypothetical_df['has_unexpired_lines_of_credit'] = True

In [241]:
mean_absolute_error(model.predict(create_input_new(hypothetical_df, trade_history_col, yield_history_cols)),
                    hypothetical_df.new_ys)

10102059.711738383

In [227]:
mean_absolute_error(model.predict(create_input_new(problem_cusips_df, trade_history_col, yield_history_cols)),
                    hypothetical_df.new_ys)

10102060.128405051

In [244]:
train_dataframe[train_dataframe.has_unexpired_lines_of_credit]

,rtrs_control_number,cusip,yield,is_callable,refund_date,accrual_date,dated_date,next_sink_date,coupon,delivery_date,...,S_min_ago_qdiff,ttypes,diff_size,abs_last_yield_spread,abs_diff_size,days_duration,trade_history_sum,moodys_rating,ratings_concat,trade_history_shortened


In [220]:
test_dataframe[test_dataframe.cusip.str[:6] == '64987D'][['cusip','new_ys', 'last_yield', 'prediction', 'error', 'coupon', 'days_to_maturity', 'default_indicator']].sort_values(by='error', key=abs)\
.groupby('cusip')['error']\
.apply(lambda x: (min(x), max(x), x.mean()))\
.apply(np.round, 2)

cusip
64987D2Y9                      [-24.0, 21.0, 1.0]
64987D3V4                      [12.0, 21.0, 15.0]
64987D5T7                    [-19.0, -4.0, -12.0]
64987D5W0                   [-41.0, -41.0, -41.0]
64987D5X8                    [-86.0, 20.0, -23.0]
64987D5Y6                   [-20.0, -20.0, -20.0]
64987D6B5                      [-13.0, 9.0, -0.0]
64987D6D1                       [-2.0, 26.0, 4.0]
64987D6G4                      [24.0, 24.0, 24.0]
64987DAW4                     [-18.0, 10.0, -0.0]
64987DEC4                       [-6.0, 24.0, 2.0]
64987DGL2                         [2.0, 2.0, 2.0]
64987DGY4                      [-17.0, 31.0, 5.0]
64987DHP2    [10102010.0, 10102104.0, 10102060.0]
64987DHZ0    [10102058.0, 10102068.0, 10102063.0]
64987DJC9    [10102035.0, 10102088.0, 10102059.0]
64987DJP0                        [-0.0, 5.0, 2.0]
64987DJW5                      [-7.0, -5.0, -7.0]
64987DK45                      [-0.0, 28.0, 11.0]
64987DKC7                      [17.0, 19.0, 

# Archive

In [54]:
#CURRENT TRIAl 

#75 EPOCHS 
#no RESIDUAL 
#DROPOUT 0.1
#GLU for ref model, dropout after GLU

t = 0
for i, prediction in enumerate(predictions):
    print(f'Trial {i+1}: {mean_absolute_error(prediction, y_test):.3f}')
    t += mean_absolute_error(prediction, y_test)
print(f'Average MAE: {t/len(predictions):.3f}')
print(f'Epochs {[len(hist.history["loss"]) for hist, _ in results]}')

Trial 1: 8.114
Trial 2: 8.234
Trial 3: 8.186
Trial 4: 8.220
Average MAE: 8.188
Epochs [98, 125, 125, 125]


In [40]:
#CURRENT TRIAl 

#75 EPOCHS 
#no RESIDUAL 
#DROPOUT 0.1
#GLU for ref model, dropout after GLU

t = 0
for i, prediction in enumerate(predictions):
    print(f'Trial {i+1}: {mean_absolute_error(prediction, y_test):.3f}')
    t += mean_absolute_error(prediction, y_test)
print(f'Average MAE: {t/len(predictions):.3f}')
print(f'Epochs {[len(hist.history["loss"]) for hist, _ in results]}')

Trial 1: 8.489
Trial 2: 8.276
Trial 3: 8.338
Trial 4: 8.403
Average MAE: 8.376
Epochs [75, 67, 75, 75]


In [36]:
#CURRENT TRIAl 

#75 EPOCHS 
#no RESIDUAL 
#DROPOUT 0.1
#GLU for both ref and hist model, no residual

t = 0
for i, prediction in enumerate(predictions):
    print(f'Trial {i+1}: {mean_absolute_error(prediction, y_test):.3f}')
    t += mean_absolute_error(prediction, y_test)
print(f'Average MAE: {t/len(predictions):.3f}')
print(f'Epochs {[len(hist.history["loss"]) for hist, _ in results]}')

Trial 1: 8.301
Trial 2: 8.174
Average MAE: 8.237
Epochs [68, 46]


In [45]:
#CURRENT TRIAl 

#75 EPOCHS 
#no RESIDUAL 
#DROPOUT 0.1
#GLU for both ref and hist model, no residual

t = 0
for i, prediction in enumerate(predictions):
    print(f'Trial {i+1}: {mean_absolute_error(prediction, y_test):.3f}')
    t += mean_absolute_error(prediction, y_test)
print(f'Average MAE: {t/len(predictions):.3f}')
print(f'Epochs {[len(hist.history["loss"]) for hist, _ in results]}')

Trial 1: 8.346
Trial 2: 8.198
Trial 3: 8.327
Trial 4: 8.215
Average MAE: 8.271
Epochs [75, 70, 75, 71]


In [32]:
#CURRENT TRIAl 

#75 EPOCHS 
#no RESIDUAL 
#DROPOUT 0.1
#GLU for ref model only with residuals ADDED

t = 0
for i, prediction in enumerate(predictions):
    print(f'Trial {i+1}: {mean_absolute_error(prediction, y_test):.3f}')
    t += mean_absolute_error(prediction, y_test)
print(f'Average MAE: {t/len(predictions):.3f}')
print(f'Epochs {[len(hist.history["loss"]) for hist, _ in results]}')

Trial 1: 8.159
Trial 2: 8.135
Trial 3: 8.235
Trial 4: 8.279
Average MAE: 8.202
Epochs [49, 49, 97, 125]


In [1088]:
#CURRENT TRIAl 

#75 EPOCHS 
#no RESIDUAL 
#DROPOUT 0.1
#GLU for ref model only with no residuals

t = 0
for i, prediction in enumerate(predictions):
    print(f'Trial {i+1}: {mean_absolute_error(prediction, y_test):.3f}')
    t += mean_absolute_error(prediction, y_test)
print(f'Average MAE: {t/len(predictions):.3f}')
print(f'Epochs {[len(hist.history["loss"]) for hist, _ in results]}')

Trial 1: 8.048
Trial 2: 8.082
Trial 3: 8.067
Trial 4: 8.061
Average MAE: 8.065
Epochs [70, 74, 68, 86]
